In [1]:
!ls

DNN_combine.pth  models		__pycache__	run_demo.py
DNN.py		 plots_demo.py	run_demo.ipynb	Save_GP_NN_Surrogates.ipynb


In [ ]:
import os
import re
import torch
import numpy as np
import pandas as pd
import warnings
import sys
import random
import pickle
sys.path.append('../../')
warnings.filterwarnings("ignore")
random.seed(10)

from DNN import Stress_resistivity_Model as Model
from src.functions import latin_hypercube_2d_uniform
from src.kernels import rbf
from src.acquisition_func import ucb
from src.gaussian_process import Gaussain_Process
from src.bayesopt import Bayesopt
from src.optimizers import tune_hyperparam_parallel
from stress_optimisation.load_exp_data import stress_resist_data_loader, append_new_data

In [ ]:
# required for scaling don't change
# (min max) pairs in each dimension
parameter_space = np.array([[2, 23], [50, 750]])

explore_K = 0.5
Ninit_samples = 15
RESULT_DIR = "/home/ashriva/work/Results/bayesopts/Bayes_opt_demo/GP_i"+str(Ninit_samples)+"k"+str(explore_K)+"/"

isExist = os.path.exists(RESULT_DIR)
if not isExist:
    os.makedirs(RESULT_DIR)

In [ ]:
def stress_obj(stress, stress_grad, resistivity):

    #     def Relu(x):
    #         return x * (x > 0)

    def modified_sigmoid(x, d):

        x = d - x
        return 1 / (1 + np.exp(-5 * x))

    def cutoff_func(x, d):

        def sigmoid(x):

            return 1 / (1 + np.exp(-x))

        f = sigmoid(d + x) + sigmoid(d - x)
        f = (f - 1)

        return f

    def pos_grad_func(x):

        return (np.tanh(x) + 1) / 2

    def mingrad_func(x):

        return (-x / 1900 + 1)

    min_grad_criteria = mingrad_func(stress_grad)
    pos_grad_criteria = pos_grad_func(stress_grad)
    stress_cutoff = cutoff_func(stress, 300)
    resistivity_cutoff = modified_sigmoid(resistivity, 3)

    switch = pos_grad_criteria * stress_cutoff * resistivity_cutoff
    obj = min_grad_criteria * switch

    return obj, min_grad_criteria, pos_grad_criteria, stress_cutoff, resistivity_cutoff

In [ ]:
class GP_surrogate_model:

    def __init__(self):

        with open('demo_gp_objects.pkl', 'rb') as file:
            data = pickle.load(file)
            self.gp_stress = data['stress']
            self.gp_resist = data['resist']

    def __call__(self, x):

        y1, _ = self.gp_stress.posterior(x)
        # y1 += .1 * np.random.normal(0, 1, size=y1.shape)

        y2, _ = self.gp_resist.posterior(x)
        # y2 += .1 * np.random.normal(0, 1, size=y2.shape)

        return y1, y2

# Initial samples uusing LHS sampling

In [ ]:
x_train = latin_hypercube_2d_uniform(
    parameter_space[0], parameter_space[1], Ninit_samples)
stress_y_train, resist_y_train = blackbox_func(x_train)

# Bayesopt setup

In [ ]:
# [sigma, l_pressure, l_power]
l_stress = [1, 0.3, 0.45]
l_resist = l_stress
l_obj = l_resist

# Gaussian process interpolator for stress
kernel_stress = rbf(l_stress)
GP_stress = Gaussain_Process(kernel_stress, parameter_space)
GP_stress.noise = 0.12

# # Gaussian process interpolator for resistance
kernel_resist = rbf(l_resist)
GP_resist = Gaussain_Process(kernel_resist, parameter_space)
GP_resist.noise = 0.058

# Create Gausian process for desired objective function
kernel_obj = rbf(l_obj)
GP_obj = Gaussain_Process(kernel_obj, parameter_space)
GP_obj.noise = 1e-3

# Create blackbox function for quering stress
blackbox_func = GP_surrogate_model()

# Define Bayesopt
bopt = Bayesopt()
search_bounds = parameter_space
acq_func = ucb(GP_obj)
acq_func.k = explore_K


# Run experiments

In [ ]:
GP_stress.reset()
GP_stress.add_observation(x_train, stress_y_train, scale=True)
GP_stress.fit()

GP_resist.reset()
GP_resist.add_observation(x_train, resist_y_train, scale=True)
GP_resist.fit()

GP_obj.reset()
_, _, stress_mu_grad_ = GP_stress.posterior(x_train, calc_grad=True)
_, _ = GP_resist.posterior(x_train)
obj_val, _, _, _, _ = stress_obj(stress_y_train, stress_mu_grad_, resist_y_train)
GP_obj.add_observation(x_train, obj_val, scale=True)
GP_obj.fit()

# Bayesopt
# Suggest next point
x_next, acq_val = bopt.next_point(acq_func, search_bounds)
print((x_next[0], round(x_next[1], 3)), acq_val)
x_next = x_next.reshape(1, -1)

In [ ]:
# Query data from the models
stress_next, resist_next = blackbox_func(x_next)
stress_next = stress_next.reshape(1, -1)
resist_next = resist_next.reshape(1, -1)

# Update to existing data
x_train = np.append(x_train, x_next, axis=0)
resist_y_train = np.append(resist_y_train, stress_next, axis=0)
stress_y_train = np.append(stress_y_train, stress_next, axis=0)